# Preprocessing
Naive bayes will be the base model, and it will struggle with stop words and common words. For that reason, we need to preprocess the data.

preprocessing steps include
1. cleaning extra newline characters
2. remove accented characters to ASCII
3. Expand contractions
4. Lowercase(?) text
5. Convert numbers to words
6. Remove numbers
7. Remove stop words
8. Lemmatization

There are a variety of python packages that will help with these steps. They are

# Table of packages here

In [1]:
# imports
import requests
import pandas as pd
import os

In [2]:
os.chdir('..')
os.getcwd()


'/home/denny/Documents/mids/w266_NLP/lit-shazam'

In [3]:
# get data
from src.data.make_dataset import get_book
# from data.raw.book_urls import book_urls # not working?
book_urls = {'great_gatsby':'https://www.gutenberg.org/cache/epub/64317/pg64317.txt',
'the_sun_also_rises':'https://www.gutenberg.org/cache/epub/67138/pg67138.txt',
'a_tale_of_two_cities':'https://www.gutenberg.org/cache/epub/98/pg98.txt'}


In [4]:
data_set = {}
for title, url in book_urls.items():
    print(url)
    data_set[title] = get_book(url)

https://www.gutenberg.org/cache/epub/64317/pg64317.txt
https://www.gutenberg.org/cache/epub/67138/pg67138.txt
https://www.gutenberg.org/cache/epub/98/pg98.txt


# Bookends
Books in project gutenberg have lots of extra text at the end of the text file. Most of this is legalese and terms of use information and this is unneeded for the language modeling.

Let's look at some examples and then create a framework to remove it from the text

In [70]:
great_gatsby = data_set['great_gatsby']

In [71]:
great_gatsby[-9000:-5000]

' 1.F.3, a full refund of\r\n        any money paid for a work or a replacement copy, if a defect in the\r\n        electronic work is discovered and reported to you within 90 days of\r\n        receipt of the work.\r\n    \r\n    • You comply with all other terms of this agreement for free\r\n        distribution of Project Gutenberg™ works.\r\n    \r\n\r\n1.E.9. If you wish to charge a fee or distribute a Project\r\nGutenberg™ electronic work or group of works on different terms than\r\nare set forth in this agreement, you must obtain permission in writing\r\nfrom the Project Gutenberg Literary Archive Foundation, the manager of\r\nthe Project Gutenberg™ trademark. Contact the Foundation as set\r\nforth in Section 3 below.\r\n\r\n1.F.\r\n\r\n1.F.1. Project Gutenberg volunteers and employees expend considerable\r\neffort to identify, do copyright research on, transcribe and proofread\r\nworks not protected by U.S. copyright law in creating the Project\r\nGutenberg™ collection. Despite

In [72]:
# end pattern
print('*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***')
end_gutenberg = '*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***'

*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***


In [75]:
pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
p = re.compile(pattern)
print('the span of the found text is ', re.search(pattern, great_gatsby).span())
print('the start of the found text is ', re.search(pattern, great_gatsby).start())
print('we should remove everything after the start fo the end of book pattern')
re.search(pattern, great_gatsby).start()

the span of the found text is  (277737, 277796)
the start of the found text is  277737
we should remove everything after the start fo the end of book pattern


277737

In [ ]:
# all books in project gutenberg end with 
# *** END OF THE PROJECT GUTENBERG EBOOK {Title} ***
# *** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***

import re
def remove_bookend(book:str)->str:
    """removes the extra end of the book in project gutenberg"""
    end_of_book_pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
    match = re.search(end_of_book_pattern, book)
    if match is None:
        print('could not find project gutenberg ending')
        raise ValueError
    last_character = match.start()
    return book[:last_character]

In [8]:
# questions
# what are the most common words?
# what words are most commonly associated with an author


In [9]:
# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    great_gatsby = great_gatsby.replace(char, " ")

In [12]:
# remove header, footer from gutenberg
great_gatsby[:1000]

'\ufeffThe Project Gutenberg eBook of The Great Gatsby        This ebook is for the use of anyone anywhere in the United States and  most other parts of the world at no cost and with almost no restrictions  whatsoever. You may copy it, give it away or re-use it under the terms  of the Project Gutenberg License included with this ebook or online  at www.gutenberg.org. If you are not located in the United States,  you will have to check the laws of the country where you are located  before using this eBook.    Title: The Great Gatsby      Author: F. Scott Fitzgerald    Release date: January 17, 2021 [eBook #64317]    Language: English        *** START OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear 

In [33]:
import re
?re.search
p = re.compile(r' START OF THE PROJECT GUTENBERG EBOOK ')
p = re.compile('The Project Gutenberg eBook')
?re.search
re.search('hi','hello there hi babe')
re.search(' START OF THE PROJECT GUTENBERG EBOOK ', great_gatsby)

<re.Match object; span=(645, 683), match=' START OF THE PROJECT GUTENBERG EBOOK '>

In [42]:
# end
print('*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***')
end_gutenberg = '*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***'

*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***


In [63]:
pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
p = re.compile(pattern)
re.search(pattern, great_gatsby).start()

277737

In [64]:
# all books in project gutenberg end with 
# *** END OF THE PROJECT GUTENBERG EBOOK {Title} ***
# *** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***

import re
def remove_bookend(book:str)->str:
    """removes the extra end of the book in project gutenberg"""
    end_of_book_pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
    match = re.search(end_of_book_pattern, book)
    if match is None:
        print('could not find project gutenberg ending')
        raise ValueError
    last_character = match.start()
    return book[:last_character]

In [69]:
test_book_ending ='This is the end of the book. *** END OF THE PROJECT GUTENBERG EBOOK THE the book title with number 10 ***'
test_book_no_ending = remove_bookend(test_book_ending)
assert test_book_no_ending == 'This is the end of the book. '

In [66]:
x = remove_bookend(great_gatsby)
x[-1000:]

'ath in the presence of this continent, compelled into an aesthetic  contemplation he neither understood nor desired, face to face for the  last time in history with something commensurate to his capacity for  wonder.    And as I sat there brooding on the old, unknown world, I thought of  Gatsby’s wonder when he first picked out the green light at the end of  Daisy’s dock. He had come a long way to this blue lawn, and his dream  must have seemed so close that he could hardly fail to grasp it. He  did not know that it was already behind him, somewhere back in that  vast obscurity beyond the city, where the dark fields of the republic  rolled on under the night.    Gatsby believed in the green light, the orgastic future that year by  year recedes before us. It eluded us then, but that’s no  matter—tomorrow we will run faster, stretch out our arms further … And  one fine morning—    So we beat on, boats against the current, borne back ceaselessly into  the past.                           

In [62]:
great_gatsby[277737 :277796]

'*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***'

In [10]:
# Normalization
# converting to the same case, stemming
great_gatsby[:50]

'\ufeffThe Project Gutenberg eBook of The Great Gatsby  '